In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import StandardScaler
from bayes_opt import BayesianOptimization
from sklearn.svm import SVR
import random
import random
import numpy as np
from sklearn.model_selection import train_test_split , KFold
from sklearn.linear_model import LinearRegression
import sklearn
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import tensorflow as tf

In [2]:


# Step 1: Load your dataset. Make sure you have a CSV file with relevant features and effort (target) column.
# Replace 'your_dataset.csv' with your actual dataset file name and adjust the column names accordingly.
maxwell=pd.read_csv("C:\\Users\\Asus\\Desktop\\Tehran university\\Seminar\\Datasets\\maxwell_dataset.csv",header=None)
columns_maxwell=['Syear','App','Har','Dba','Ifc','Source','Telonuse','Nlan','T01','T02','T03','T04','T05','T06','T07','T08','T09','T10','T11','T12','T13','T14','T15','Duration','Size','Time','Effort']
maxwell.set_axis(columns_maxwell,axis='columns',inplace=True)
maxwell.set_axis(range(1,63),axis=0 ,inplace=True)
maxwell.rename_axis("Features", axis=1,inplace=True)
maxwell.rename_axis("Projects", axis=0,inplace=True)
dataset = maxwell

# Step 2: Prepare the data.
X = dataset.drop(columns=['Effort']).values  # Features
y = dataset['Effort'].values  # Target (effort)

seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)


#=========================================================================================================================
# Define the fitness function
def fitness_function(subset):
    selected_features = [feature for feature, is_selected in zip(range(X.shape[1]), subset) if is_selected]
    if len(selected_features) == 0:
        return float('-inf')  # Penalize subsets with no selected features
    
    num_folds = 5
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed_value)
    
    X_selected = X[:, selected_features]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_selected)
    error_list=[]
    
    for train_index, val_index in kf.split(X_scaled):
        X_train, X_test = X_scaled[train_index], X_scaled[val_index]
        y_train, y_test = y[train_index], y[val_index]
    
    
        # Step 3: Build the ANN model.
        model = SVR(kernel='linear')

        # Step 5: Train the model.
        model.fit(X_train, y_train)

        # Step 6: Evaluate the model.
        y_pred = model.predict(X_test)
    
        error = np.mean(np.abs(y_pred - y_test))
        error_list.append(error)
        
    return 1 / (1 + np.mean(error_list)) 
#========================================================================================================================

max_fitness_score=0
max_score_features=0

# Genetic Algorithm Parameters
population_size = 10
num_generations = 100
crossover_rate = 0.8
mutation_rate = 0.1

# Create an initial population of feature subsets
population = []
for _ in range(population_size):
    subset = [random.randint(0, 1) for _ in range(X.shape[1])]
    population.append(subset)
    
# Genetic Algorithm
for generation in range(num_generations):
    # Evaluate fitness for each feature subset in the population
    fitness_scores = [fitness_function(subset) for subset in population]
    if max(fitness_scores) > max_fitness_score:
        max_fitness_score = max(fitness_scores)
        max_score_features =population[fitness_scores.index(max(fitness_scores))]
    
    # Selection
    selected_population = sorted(population, key=lambda x: fitness_function(x), reverse=True)
    
    # Crossover
    elitism_num = 2
    offspring_population = selected_population[:elitism_num] # Elitism, preserving the top individuals
    for i in range(0, population_size, 2):
        parent1 = selected_population[i]
        parent2 = selected_population[i + 1]
        
        if random.random() < crossover_rate:
            crossover_point = random.randint(1, len(parent1) - 1)
            child1 = parent1[:crossover_point] + parent2[crossover_point:]
            child2 = parent2[:crossover_point] + parent1[crossover_point:]
        else:
            child1 = parent1
            child2 = parent2
        
        offspring_population.append(child1)
        offspring_population.append(child2)
    
    # Mutation
    for i in range(2,population_size):
        if random.random() < mutation_rate:
            mutated_feature = random.randint(0, X.shape[1] - 1)
            offspring_population[i][mutated_feature] = 1 - offspring_population[i][mutated_feature]
    
    # Replace the old population with the new offspring population
    population = offspring_population

# Select the best feature subset from the final population
best_subset = max(population, key=fitness_function)
selected_features = [feature for feature, is_selected in zip(range(X.shape[1]), best_subset) if is_selected]


#======================================================================================================================

# Train the final model using the selected features

pbounds = {'C': (0.1, 10),
           'epsilon': (0.01, 1),
           'gamma': (0.01, 100),
           'kernel': (0, 3)  # Use indices for categorical parameter
}
kernel_mapping = {0: 'linear', 1: 'rbf', 2: 'poly', 3: 'sigmoid'}

def svr_model(kernel,C, epsilon, gamma):
    model = SVR( kernel = kernel_mapping[int(kernel)], C=C, epsilon=epsilon, gamma=gamma)

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return -rmse  # Minimize the negative RMSE for Bayesian optimization.
    
    
num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed_value)
    
X_selected = X[:, selected_features]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)

mean_MAE=[]
mean_MMRE=[]
mean_RMSE=[]

for train_index, val_index in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[val_index]
    y_train, y_test = y[train_index], y[val_index]
    
    # Perform Bayesian optimization.
    optimizer = BayesianOptimization(
        f=svr_model,
        pbounds=pbounds,
        random_state=42,
        verbose=2,
    )


    optimizer.maximize(init_points=10, n_iter=30)  # Adjust the number of initial points and iterations.

    # Print the best hyperparameters found.
    best_params = optimizer.max['params']
    print("Best Hyperparameters:")
    print(best_params)

    best_C = best_params['C']
    best_epsilon = best_params['epsilon']
    best_gamma = best_params['gamma']
    best_kernel= kernel_mapping[int(best_params['kernel'])]
    
    # Train the final SVR model with the best parameters
    final_model = SVR(C=best_C, epsilon=best_epsilon, gamma=best_gamma,kernel= best_kernel)
    final_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred = final_model.predict(X_test)
    

    mae = mean_absolute_error(y_test, y_pred)
    mean_MAE.append(mae)

    # Calculate the Root Mean Squared Error (RMSE) for this fold.
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mean_RMSE.append(rmse)

     # Convert y_pred and y_test to NumPy arrays for MMRE calculation
    y_pred = np.array(y_pred).flatten()
    y_test = np.array(y_test)
    
    # Calculate the Mean Magnitude of Relative Error (MMRE) for this fold.
    mmre = np.mean(np.abs((y_test - y_pred) / y_test))
    mean_MMRE.append(mmre)

print(f"Mean Absolute Error mean: {np.mean(mean_MAE)}")   
print(f"Mean Magnitude of Relative Error mean (MMRE): {np.mean(mean_MMRE):.2f}")
print(f"Root Mean Squared Error (RMSE) mean: {np.mean(mean_RMSE)}")
#===================================================================================================================

# train the model using max_features

# X_selected = X[:, max_score_features]
# X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# pbounds = {'C': (0.1, 10),
#            'epsilon': (0.01, 1),
#            'gamma': (0.001, 1000),
#            'kernel': (0, 3)  # Use indices for categorical parameter
# }
# kernel_mapping = {0: 'linear', 1: 'rbf', 2: 'poly', 3: 'sigmoid'}

# def svr_model(kernel,C, epsilon, gamma):
#     model = SVR( kernel = kernel_mapping[int(kernel)], C=C, epsilon=epsilon, gamma=gamma)

#     model.fit(X_train_scaled, y_train)

#     y_pred = model.predict(X_test_scaled)

#     rmse = np.sqrt(mean_squared_error(y_test, y_pred))
#     return -rmse  # Minimize the negative RMSE for Bayesian optimization.
    
    
# # Perform Bayesian optimization.
# optimizer = BayesianOptimization(
#     f=svr_model,
#     pbounds=pbounds,
#     random_state=42,
#     verbose=2,
# )


# optimizer.maximize(init_points=10, n_iter=30)  # Adjust the number of initial points and iterations.

# # Print the best hyperparameters found.
# best_params = optimizer.max['params']
# print("Best Hyperparameters:")
# print(best_params)

# best_C = best_params['C']
# best_epsilon = best_params['epsilon']
# best_gamma = best_params['gamma']
# best_kernel= kernel_mapping[int(best_params['kernel'])]

# # Train the final SVR model with the best parameters
# final_model = SVR(C=best_C, epsilon=best_epsilon, gamma=best_gamma,kernel= best_kernel)
# final_model.fit(X_train_scaled, y_train)
    
# # Make predictions on the test set
# y_pred = final_model.predict(X_test_scaled)

# # Calculate the Root Mean Squared Error (RMSE) to assess the model's performance.
# rmse3 = np.sqrt(mean_squared_error(y_test, y_pred))
# print(f"Root Mean Squared Error (RMSE): {rmse3}")

# # Convert y_pred and y_test to NumPy arrays
# y_pred = np.array(y_pred).flatten()
# y_test = np.array(y_test)

# # Calculate Mean Magnitude of Relative Error (MMRE)
# mmre = np.mean(np.abs((y_test - y_pred) / y_test))
# print(f"Mean Magnitude of Relative Error (MMRE): {mmre:.2f}")




# print(best_subset," \n ", max_score_features)




|   iter    |  target   |     C     |  epsilon  |   gamma   |  kernel   |
-------------------------------------------------------------------------
| 1         | -1.754e+0 | 3.808     | 0.9512    | 73.2      | 1.796     |
| 2         | -5.623e+0 | 1.645     | 0.1644    | 5.818     | 2.599     |
| 3         | -5.624e+0 | 6.051     | 0.711     | 2.068     | 2.91      |
| 4         | -1.678e+0 | 8.341     | 0.2202    | 18.19     | 0.5502    |
| 5         | -1.725e+0 | 3.112     | 0.5295    | 43.2      | 0.8737    |
| 6         | -1.754e+0 | 6.157     | 0.1481    | 29.22     | 1.099     |
| 7         | -1.754e+0 | 4.615     | 0.7873    | 19.98     | 1.543     |
| 8         | -1.699e+0 | 5.965     | 0.05599   | 60.76     | 0.5116    |
| 9         | -5.624e+0 | 0.744     | 0.9494    | 96.56     | 2.425     |
| 10        | -1.754e+0 | 3.116     | 0.1067    | 68.43     | 1.32      |
| 11        | -1.67e+04 | 10.0      | 0.01      | 100.0     | 0.0       |
| 12        | -1.753e+0 | 0.1       | 

| 24        | -6.84e+03 | 0.9224    | 0.8004    | 92.2      | 2.915     |
| 25        | -6.841e+0 | 3.709     | 0.2939    | 69.23     | 2.002     |
| 26        | -6.84e+03 | 9.87      | 0.609     | 90.06     | 2.271     |
| 27        | -1.057e+0 | 7.168     | 0.7501    | 4.341     | 1.208     |
| 28        | -1.057e+0 | 8.469     | 0.7207    | 29.55     | 1.622     |
| 29        | -1.057e+0 | 0.9355    | 0.8115    | 69.15     | 1.816     |
| 30        | -9.897e+0 | 7.947     | 0.9743    | 72.85     | 0.116     |
| 31        | -6.841e+0 | 2.385     | 0.4105    | 94.55     | 2.585     |
| 32        | -6.841e+0 | 0.6783    | 0.1375    | 55.04     | 2.609     |
| 33        | -1.057e+0 | 9.479     | 0.6077    | 16.88     | 1.321     |
| 34        | -1.057e+0 | 1.572     | 0.9425    | 53.51     | 1.852     |
| 35        | -6.841e+0 | 0.708     | 0.5208    | 42.71     | 2.859     |
| 36        | -1.026e+0 | 3.538     | 0.2252    | 66.69     | 0.4801    |
| 37        | -1.057e+0 | 1.675     | 

| 13        | -4.187e+0 | 6.022     | 0.6101    | 6.57      | 3.0       |
| 14        | -4.285e+0 | 9.711     | 0.7125    | 99.39     | 1.122     |
| 15        | -3.94e+03 | 4.768     | 0.1689    | 80.2      | 0.2895    |
| 16        | -2.549e+0 | 9.389     | 0.7093    | 45.02     | 2.995     |
| 17        | -4.285e+0 | 7.774     | 0.8833    | 22.72     | 1.117     |
| 18        | -2.549e+0 | 2.016     | 0.625     | 24.83     | 2.303     |
| 19        | -4.285e+0 | 0.1624    | 0.4647    | 97.67     | 1.58      |
| 20        | -3.775e+0 | 7.452     | 0.5298    | 30.04     | 0.05903   |
| 21        | -4.285e+0 | 8.882     | 0.5357    | 25.7      | 1.594     |
| 22        | -4.111e+0 | 2.141     | 0.9702    | 1.443     | 0.2084    |
| 23        | -2.549e+0 | 4.539     | 0.09703   | 5.181     | 2.511     |
| 24        | -2.549e+0 | 0.9224    | 0.8004    | 92.2      | 2.915     |
| 25        | -2.549e+0 | 3.709     | 0.2939    | 69.23     | 2.002     |
| 26        | -2.549e+0 | 3.714     | 

| 38        | -7.993e+0 | 1.021     | 0.7537    | 65.63     | 2.533     |
| 39        | -1.077e+0 | 7.191     | 0.06195   | 47.45     | 0.5273    |
| 40        | -1.066e+0 | 9.01      | 0.218     | 46.14     | 0.4312    |
Best Hyperparameters:
{'C': 0.7440107705542672, 'epsilon': 0.9493966818808, 'gamma': 96.56354698712519, 'kernel': 2.4251920443493833}
Mean Absolute Error mean: 3419.981367782059
Mean Magnitude of Relative Error mean (MMRE): 1.02
Root Mean Squared Error (RMSE) mean: 4787.824222006315


In [3]:
selected_features

[1, 5, 7, 8, 9, 11, 14, 15, 16, 17, 21, 22, 23, 24]